
Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>


In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
from bs4 import BeautifulSoup

In [50]:
KEYWORDS = ['python', 'парсинг']

In [19]:
res = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text)

In [51]:
posts = soup.find_all('article', class_='tm-articles-list__item')
posts

[<article class="tm-articles-list__item" data-navigatable="" id="570710" tabindex="0"><div class="tm-article-snippet"><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" href="/ru/users/Vladimir_Kuiantsev/" title="Vladimir_Kuiantsev"><div class="tm-entity-image"><svg class="tm-svg-img tm-image-placeholder tm-image-placeholder_pink" height="24" width="24"><!-- --><use xlink:href="/img/megazord-v24.cee85629.svg#placeholder-user"></use></svg></div></a><span class="tm-user-info__user"><a class="tm-user-info__username" href="/ru/users/Vladimir_Kuiantsev/">
       Vladimir_Kuiantsev
     </a></span></span><span class="tm-article-snippet__datetime-published"><time datetime="2021-08-01T08:39:52.000Z" title="2021-08-01, 11:39">сегодня в 11:39</time></span></div><!-- --></div><h2 class="tm-article-snippet__title tm-article-snippet__title_h2"><a class="tm-article-snippet__title

In [49]:
kom_news = pd.DataFrame()
for post in posts:
    for word in KEYWORDS:
        if word in post.text.lower():
            
            date = soup.find('span', class_='tm-article-snippet__datetime-published').text
            articles_intro = soup.find('h2', class_='tm-article-snippet__title').text
            a_list = soup.find('a', class_='tm-article-snippet__title-link')
            ref = a_list.attrs.get('href')
            all_refs = 'https://habr.com' + ref
            row = {'date': date, 'title': articles_intro, 'all_refs' : all_refs}
            kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news.drop_duplicates(subset=['all_refs'], keep='first')

,date,title,all_refs
0,сегодня в 11:39,Миграция платежной платформы в облако: Зачем и...,https://habr.com/ru/post/570710/



Задание 2.
Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.


In [6]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [7]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [8]:
payload = {
'emailAddresses': EMAIL
}

In [9]:
headers = {
'Vaar-Header-App-Build-Version': '1.0.0',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
'Vaar-Version': '0'
}

In [10]:
req = requests.post(URL, json=payload, headers=headers)

In [11]:
req

<Response [200]>

In [12]:
req.json()

{'breaches': {'2': {'breachId': 2,
   'site': 'linkedin.com',
   'recordsCount': 158591429,
   'description': "In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.",
   'publishDate': '2016-10-21T00:00:00Z',
   'statistics': {'usernames': 0,
    'passwords': 111975337,
    'emails': 158591429}},
  '3': {'breachId': 3,
   'site': 'adobe.com',
   'recordsCount': 152046506,
   'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software produc

In [12]:
avast = pd.DataFrame(req.json()['breaches'])

In [17]:
row1 = avast.iloc[[4, 1, 3]] 
row2 = row1.T
row2

,publishDate,site,description
2,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
3,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
12,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
15,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
41,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
2961,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
3164,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
3176,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3520,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
3587,2017-03-01T00:00:00Z,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."
